# Analysis of Appendix B.3.2

In [ ]:
library(dplyr)
library(ggplot2)
library(ggpubr)
source("helper.r")
theme_set(theme_pubr(legend = "none"))

### Data Preparation 

In [ ]:
# path to folder, with folders for dataset results
path = "../data/runs/mlp/"
datasets = list.files(path)

# create list with one list containing one dataframe per dataset
data.list = lapply(datasets, function(data){
  
  data.path = paste0(path, data, "/2_3_effects_and_trees/")
  objectives = list.files(data.path)
  
  for(i in 1:length(objectives)){
 
    res = readRDS(paste0(data.path, objectives[i]))
    df.sub = res$result[[1]]$eval
    df.sub$objective = res$objective
    
    if(i == 1) df = df.sub
    else df = rbind(df, df.sub)
  }
  
  return(df)
})
names(data.list) = datasets

### Create Table  7

In [ ]:
# relative confidence improvement on dataset level
df.datasets.conf = create_table_datasets(data.list, "SS_L2", "conf.rel", 7)

# count features with highest and lowest improvements (for Table 7)
df.feat.low      = as.data.frame(df.datasets.conf %>% group_by(feature.1) %>% summarise(n()))
df.feat.high     = as.data.frame(df.datasets.conf %>% group_by(feature) %>% summarise(n()))
df.feat.conf     = left_join(df.feat.high, df.feat.low, by = c("feature"= "feature.1"))

df.datasets.conf[,c("mean","sd","mean.1", "mean.2")] = round(df.datasets.conf[,c("mean","sd","mean.1", "mean.2")]*100,0)
names(df.datasets.conf)[4:7] = c("feat.high", "mean.high", "feat.low", "mean.low")
df.datasets.conf[,c(1:3,5,7)]

### Create Table 8

In [ ]:
# relative negative loglikelihood improvement on dataset level
df.datasets.nll  = create_table_datasets(data.list, "SS_L2", "neg_loglik.rel", 7)

# count features with highest and lowest improvements (for Table 7)
df.feat.low      = as.data.frame(df.datasets.nll %>% group_by(feature.1) %>% summarise(n()))
df.feat.high     = as.data.frame(df.datasets.nll %>% group_by(feature) %>% summarise(n()))
df.feat.nll      = left_join(df.feat.high, df.feat.low, by = c("feature"= "feature.1"))

df.datasets.nll[,c("mean","sd","mean.1", "mean.2")] = round(df.datasets.nll[,c("mean","sd","mean.1", "mean.2")]*100,0)
names(df.datasets.nll)[4:7] = c("feat.high", "mean.high", "feat.low", "mean.low")
df.datasets.nll[,c(1:3,5,7)]


### Create Table 9

In [ ]:
df.feat = left_join(df.feat.conf, df.feat.nll, by = "feature")
names(df.feat) = c("hyperparameter","#MC.high", "#MC.low", "#NLL.high", "#NLL.low") 
df.feat

## Analyse different objective functions 

### Create Table 10

In [ ]:
targets = c("conf.rel", "conf.rel.opt.1", "neg_loglik.rel")
tab.obj = lapply(targets, function(target){
            tab.l2 = create_table_features(data.list = data.list, objective = "SS_L2", target = target, depth = 7)[,1:2]
            tab.ar = create_table_features(data.list = data.list, objective = "SS_area", target = target, depth = 7)[,1:2]
            tab.sd = create_table_features(data.list = data.list, objective = "SS_sd", target = target, depth = 7)[,1:2]
            tab.mu = create_table_features(data.list = data.list, objective = "SS_mean", target = target, depth = 7)[,1:2]
            tab = left_join(left_join(tab.l2, tab.ar, by = "feature"), tab.sd, by = "feature")
            tab[,2:5] = tab[,2:5]*100
            names(tab) = c("hyperparameter", "l2", "area", "sd". "mean")
            tab
        })
names(tab.obj) = targets
tab.obj


### Increased Confidence with more splits

#### Prepare data to plot

In [ ]:
# get data for CNAE-9 dataset for run 9
path = "../data/runs/mlp/"
dataset   = "cnae-9"
iter      = 9
data.all  = readRDS(paste0(path, dataset, "/2_3_effects_and_trees/eval_SS_L2_20_1000.rds"))
testdata  = readRDS(paste0(path, dataset, "/2_1_testdata/testdata_1000.rds"))
features  = unique(data.all$result[[1]]$feature)
optima    = readRDS(paste0(path, dataset,"/1_1_mlrmbo_runs/mlrmbo_run_lambda_1_30repls.rds" ))
traindata = optima$result[[iter]]$opt.path
optimum   = traindata[which.min(traindata$y), features]
gt        = readRDS(paste0(path, dataset, "/2_2_groundtruth_pdps/gtpdp_20_1000.rds"))
#tree_data      = readRDS(paste0(path, dataset, "/2_3_effects_and_trees/trees.rds"))



# Calculate PDP and confidence estimates for hyperparameter number of layers
feat = "num_layers"

# calculate global PDP and confidence bands
data = data.all$result[[1]]$reslist[[iter]][[feat]]
data.pdp = data$res.pdp
data.pdp$lower = data.pdp$mean - data.pdp$sd*1.96
data.pdp$upper = data.pdp$mean + data.pdp$sd*1.96

# Extract ICE curves
data.ice = data$res.ice

# Extract tree and split criteria for optimal subregions
tree = data$trees
split.criteria = find_split_criteria(tree[[1]], optimum)
split.criteria$values = round(split.criteria$values, 4)

# Extract true PDP and ICE estimates
gt.feat = gt[[1]][[feat]][[1]]
gt.pdp = gt.feat[gt.feat$.type=="pdp",]
gt.ice = gt.feat[gt.feat$.type == "ice",]

# Calculate PDP and confidence estimates for optimal subregion after last split
optimal.node = find_optimal_node(tree[[1]][1:7], optimum)
ind.opt = optimal.node$subset.idx
pdp.opt = data.ice[data.ice$.id %in% ind.opt,] %>% group_by(get(feat)) %>% summarise(mean.opt = mean(mean), sd.opt = mean(sd))
pdp.opt$lower = pdp.opt$mean.opt - pdp.opt$sd.opt*1.96
pdp.opt$upper = pdp.opt$mean.opt + pdp.opt$sd.opt*1.96
colnames(pdp.opt) = c(feat, "mean", "sd",   "lower" , "upper")
gt.opt = gt.ice[gt.ice$.id %in% ind.opt,] %>% group_by(get(feat)) %>% summarise(mean.opt = mean(mean))
colnames(gt.opt) = c(feat, "mean")

# Calculate PDP and confidence estimates for optimal subregion after second split
optimal.node = find_optimal_node(tree[[1]][1:3], optimum)
ind.opt.3 = optimal.node$subset.idx
pdp.opt.3 = data.ice[data.ice$.id %in% ind.opt.3,] %>% group_by(get(feat)) %>% summarise(mean.opt = mean(mean), sd.opt = mean(sd))
pdp.opt.3$lower = pdp.opt.3$mean.opt - pdp.opt.3$sd.opt*1.96
pdp.opt.3$upper = pdp.opt.3$mean.opt + pdp.opt.3$sd.opt*1.96
colnames(pdp.opt.3) = c(feat, "mean", "sd",   "lower" , "upper")
gt.opt.3 = gt.ice[gt.ice$.id %in% ind.opt.3,] %>% group_by(get(feat)) %>% summarise(mean.opt = mean(mean))
colnames(gt.opt.3) = c(feat, "mean")



#### Create Figure 9

In [ ]:
y_lim = c(-0.2, 0.8)

# Global PDP with confidence bands
p.pdp = ggplot(data = data.pdp, aes_string(x = feat, y = "mean")) + 
  geom_line(colour = "blue", lwd = 1) + 
  geom_ribbon(aes(ymin = lower, ymax = upper), alpha = 0.2) + 
  geom_line(data = gt.pdp, lwd = 1) +
  geom_vline(aes(xintercept = as.numeric(optimum[feat])), colour = "orange", lwd = 1) +
  ylim(y_lim) + xlab("Num. Layers") + ylab("c") +
  labs(title = paste0("Entire HP Space, n = ", 1000)) +
  theme(plot.caption = element_text(hjust = 0, face= "italic", size = 11),
        plot.title = element_text(size = 11),
        axis.title.y = element_text(size = 14))

# Sub-regional PDP with confidence bands after split 6
p.pdp.opt = ggplot(data = pdp.opt, aes_string(x = feat, y = "mean")) + 
  geom_line(colour = "blue", lwd = 1) + 
  geom_ribbon(aes(ymin = lower, ymax = upper), alpha = 0.2) + 
  geom_line(data = gt.opt, lwd = 1) +
  geom_vline(aes(xintercept = as.numeric(optimum[feat])), colour = "orange", lwd = 1) +
  xlab("Num. Layers") + ylim(y_lim) + ylab("c")  + 
  labs(title = paste0("Split 6: Opt. Sub-region, n = ", length(ind.opt))) +
  theme(plot.caption = element_text(hjust = 0, face= "italic", size = 11),
        plot.title = element_text(size = 11),
        axis.title.y = element_text(size = 14))

# Sub-regional PDP with confidence bands after split 2
p.pdp.opt.3 = ggplot(data = pdp.opt.3, aes_string(x = feat, y = "mean")) + 
  geom_line(colour = "blue", lwd = 1) + 
  geom_ribbon(aes(ymin = lower, ymax = upper), alpha = 0.2) + 
  geom_line(data = gt.opt.3, lwd = 1) +
  geom_vline(aes(xintercept = as.numeric(optimum[feat])), colour = "orange", lwd = 1) +
  xlab("Num. Layers") + ylim(y_lim) + ylab("c")  + 
  labs(title = paste0("Split 2: Opt. Sub-region, n = ", length(ind.opt.3))) +
  theme(plot.caption = element_text(hjust = 0, face= "italic", size = 11),
        plot.title = element_text(size = 11),
        axis.title.y = element_text(size = 14))

gridExtra::grid.arrange(p.pdp, p.pdp.opt.3, p.pdp.opt, nrow = 1)

